In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999


import time
import unicodedata
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, WebDriverException

import datetime
from datetime import timedelta, datetime

import csv
import os

import sys
sys.path.append('./lib/')
from functions import *
from tqdm import tnrange, tqdm_notebook
from unidecode import unidecode

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import lxml.html
import lxml

import glob

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("darkgrid")

/Users/jaime/anaconda3/lib/python3.6/site-packages/numexpr/cpuinfo.py:53: UserWarning: [Errno 20] Not a directory
  stacklevel=stacklevel + 1)
/Users/jaime/anaconda3/lib/python3.6/site-packages/numexpr/cpuinfo.py:53: UserWarning: [Errno 20] Not a directory
  stacklevel=stacklevel + 1)
/Users/jaime/anaconda3/lib/python3.6/site-packages/numexpr/cpuinfo.py:76: UserWarning: [Errno 20] Not a directory
  stacklevel=stacklevel + 1):


# Import data

In [2]:
col_names = ['entity', 'department', 'contract', 'year', 'month', 'group', 'lastn', 'lastn2', 'givenn', 'degree', 
             'qualif', 'position', 'region', 'specials', 'currency', 'salary', 'overt', 'start_date', 'end_date', 'obs', 'url', 'other']
len(col_names)

22

In [3]:
period = '20180210-3'

In [9]:
file = '../data/output/scraped_data_{}.csv'.format(period)
dfscraped = pd.read_csv(file, low_memory=False, delimiter='|', quoting=csv.QUOTE_ALL, names=col_names)

In [10]:
dfscraped['filename'] = file

In [11]:
dfscraped.filename.value_counts()

../data/output/scraped_data_20180210-3.csv    2366454
Name: filename, dtype: int64

In [12]:
dfscraped.columns

Index(['entity', 'department', 'contract', 'year', 'month', 'group', 'lastn',
       'lastn2', 'givenn', 'degree', 'qualif', 'position', 'region',
       'specials', 'currency', 'salary', 'overt', 'start_date', 'end_date',
       'obs', 'url', 'other', 'filename'],
      dtype='object')

In [14]:
dfscraped.head(2)

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,qualif,position,region,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename
0,Presidencia de la República,Presidencia de la República,Dotación de planta,Año 2017,allyear,AUTORIDAD DE GOBIERNO,BACHELET,JERIA,VERONICA MICHELLE,NaN,MÉDICO,PRESIDENTA DE LA REPÚBLICA,RM,"2, 4, 8, 9",PESOS,10.168.106,No,11/03/2014,30/11/-1,NaN,http://www.gobiernotransparentechile.cl/direct...,NaN,../data/output/scraped_data_20180210-3.csv
1,Presidencia de la República,Presidencia de la República,Dotación de planta,Año 2017,allyear,DIRECTIVO,SANDOVAL,NAVARRETE,OSCAR OMAR,NaN,ABOGADO,"DIRECTOR ADMINISTRATIVO, DIRECCIÓN ADMINISTRATIVA",RM,"5, 7, 8, 9",PESOS,7.959.883,No,01/04/2014,30/11/-1,FUNCIÓN CRÍTICA = 100%,http://www.gobiernotransparentechile.cl/direct...,NaN,../data/output/scraped_data_20180210-3.csv


In [25]:
pd.notnull(dfscraped['other']).value_counts()

False    2308040
True       58414
Name: other, dtype: int64

In [16]:
dfscraped.entity.value_counts()

Ministerio del Interior y Seguridad Pública        682791
Ministerio de Obras Públicas                       298220
Ministerio de Hacienda                             194651
Ministerio de Justicia y Derechos Humanos          181450
Ministerio de Educación                            180334
Ministerio de Vivienda y Urbanismo                 171786
Ministerio de Economía, Fomento y Turismo          132745
Ministerio del Trabajo y Previsión Social          129472
Ministerio de Desarrollo Social                    106972
Consejo de Defensa del Estado                       59255
Ministerio de Defensa Nacional                      47427
Ministerio de Minería                               45374
Ministerio de Agricultura                           44597
Ministerio de Relaciones Exteriores                 25493
Ministerio Secretaría General de la Presidencia     14657
Presidencia de la República                         13062
Ministerio del Medio Ambiente                        7227
Ministerio del

In [17]:
dfscraped.url.nunique()

36466

# Data that was not captured with the right encoding, scrape again

In [20]:
retake_entity = list(dfscraped[dfscraped.entity.str.contains("b'", na=False)].entity.value_counts().index)
retake_entity = retake_entity + list(dfscraped[dfscraped.entity.str.contains("Ã", na=False)].entity.value_counts().index)
retake_entity = retake_entity + list(dfscraped[dfscraped.entity.str.contains("Ã­", na=False)].entity.value_counts().index)
retake_entity = retake_entity + ['Ministerio del Interior y Seguridad Pblica', 'Presidencia de la Repblica ']  

In [21]:
print('Total Urls', dfscraped.url.nunique())
print('Urls to retake', dfscraped[dfscraped.entity.isin(retake_entity)].url.nunique())

Total Urls 36466
Urls to retake 0


In [22]:
retake_entity_urls = []
for u in list(set(dfscraped.loc[dfscraped.entity.isin(retake_entity), 'url'].unique())):
    if str(u) != 'nan':
        if "b'" in u:
            u = u[2:]
        if u[-1] == "'":
            u = u[:len(u)-1]
        retake_entity_urls.append(u)
len(retake_entity_urls)

0

In [23]:
retake_entity_urls

[]

### Open browser and re-scrape data 

In [20]:
output_file = '..data/output/retake_entity_0.csv'

In [22]:
if not os.path.isfile(output_file):
    browser = webdriver.Chrome()
    for url in retake_entity_urls[0:5]:
        functions.getTableData2(output_file, url, browser)
    

In [26]:
try:
    retake_entity_df = pd.read_csv(output_file, encoding='latin1', names=col_names)
except:
    retake_entity_df = pd.DataFrame()

In [28]:
try:
    retake_entity_df.entity.value_counts()
except:
    pass

In [29]:
cleanLatin(retake_entity_df)

A Jupyter Widget

### Delete the corresponding rows and add the new ones 

In [30]:
retake_entity_df['filename'] = output_file
print(retake_entity_df.shape)
print(dfscraped[dfscraped.entity.isin(retake_entity)].shape)
dfscraped = dfscraped[~dfscraped.entity.isin(retake_entity)]
dfscraped = pd.concat([dfscraped, retake_entity_df])
dfscraped.shape

(0, 1)
(0, 23)


(906540, 23)

# Transformations 

## Write or Read clean file, wrong characters, duplicate rows

In [26]:
write_clean_file = True
apply_cleanup_again = False


In [27]:
if write_clean_file:
    print('Before Processing:', dfscraped.shape)
    columns_dupe = [col for col in dfscraped.columns if not 'filename' in col]
    cleanLatin(dfscraped)

Before Processing: (2366454, 23)


A Jupyter Widget

In [28]:
dfscraped1 = dfscraped
# Lower case values
dfscraped1.givenn = dfscraped1.givenn.str.lower()
dfscraped1.lastn = dfscraped1.lastn.str.lower()
dfscraped1.lastn2 = dfscraped1.lastn2.str.lower()

dfscraped1 = dfscraped.replace(np.nan, -1).drop_duplicates(columns_dupe) # Result used to be 1693274, with no replace()
print('After Processing:', dfscraped1.shape)


# Write file
dfscraped1.to_csv('../data/output/scraped_data_clean_{}.csv'.format(period), encoding='utf-8', index=False)
    

After Processing: (1954998, 23)


In [29]:
if not write_clean_file:
    dfscraped1 = pd.read_csv('./output/scraped_data_clean.csv', encoding='latin1', low_memory=False)
    columns_dupe = [col for col in dfscraped1.columns if not 'filename' in col]

if apply_cleanup_again:
    cleanLatin(dfscraped1)
    dfscraped1.drop_duplicates(columns_dupe, inplace=True)

## Contract flipped with Department

In [73]:
dfscraped1['auxDept'] = [dept if 'per' not in dept else cont for (dept,cont) in zip(dfscraped1['department'], dfscraped1['contract'])]
dfscraped1['auxCont'] = [cont if 'per' in cont else dept for (dept,cont) in zip(dfscraped1['department'], dfscraped1['contract'])]

dfscraped1['department'] = dfscraped1['auxDept']
dfscraped1['contract'] = dfscraped1['auxCont']

del dfscraped1['auxCont']
del dfscraped1['auxDept']

## Dates 

### Format text 

In [60]:
#Format
dfscraped1['year'] = dfscraped1['year'].str.lower()
dfscraped1['month'] = dfscraped1['month'].str.lower()

#Combine
dfscraped1['yearmonth'] = dfscraped1['year'] + '-' + dfscraped1['month']

# Remove keywords
dfscraped1['yearmonth'] = dfscraped1['yearmonth'].str.replace('año ', '')
dfscraped1['yearmonth'] = dfscraped1['yearmonth'].str.replace('a�o ', '')

# Allyear will be considered january

### Cleaning allyear 

In [61]:
dfscraped1.loc[dfscraped1['yearmonth'].str.contains('allyear'), 'yearmonth'].value_counts()

2008-allyear                                                47155
2007-allyear                                                44164
2011-allyear                                                41896
2010-allyear                                                41437
2014-allyear                                                39177
2006-allyear                                                39044
2013-allyear                                                37960
2012-allyear                                                37943
2009-allyear                                                36113
2015-allyear                                                35799
2016-allyear                                                28728
2017-allyear                                                25332
agosto-allyear                                              13159
junio-allyear                                               12841
julio-allyear                                               12716
febrero-al

In [75]:
dfscraped1.loc[dfscraped1['yearmonth'].str.contains('agosto-allyear'), 'department'].value_counts()

Servicio Nacional de Menores (SENAME)              8601
Gendarmería de Chile                               2579
Subsecretaría de Vivienda y Urbanismo              1263
Dirección General de Crédito Prendario (DICREP)     588
Dotación a contrata                                 282
Servicio Nacional del Adulto Mayor (SENAMA)         195
Gobierno Regional de Los Ríos                        86
SERVIU XI Región                                     81
Dotación de planta                                   72
Name: department, dtype: int64

### Shifted data because of additional columns (in progress) 

In [74]:
dfscraped1.loc[dfscraped1['url'].str.contains('31/12/2011', na=False)]

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,qualif,position,region,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename,yearmonth
399550,Ministerio del Interior y Seguridad Pública,Gobernación de Los Andes,per_contrata,2011,allyear,1,profesional,correa,herrera,SERGIO HERNAN,,INGENIERIA COMERCIAL,COORDINADOR DELEGADO DEL COMPLEJO FRONTERIZO ...,5,1,Pesos,1.387.680,No,4743 0,01/01/2011,31/12/2011,,../data/output/scraped_data_20180210-3.csv,2011-allyear
399551,Ministerio del Interior y Seguridad Pública,Gobernación de Los Andes,per_contrata,2011,allyear,2,profesional,lorie,dabovic,MARCOS ANDRES,,ABOGADO,ASESOR AUTORIDAD,5,1,Pesos,1.387.680,No,4743 0,01/01/2011,31/12/2011,,../data/output/scraped_data_20180210-3.csv,2011-allyear
399552,Ministerio del Interior y Seguridad Pública,Gobernación de Los Andes,per_contrata,2011,allyear,3,profesional,rosende,bey,VERONICA CECILIA,,TRABAJADOR SOCIAL,PROFESIONAL AREA SOCIAL,5,1,Pesos,1.387.679,No,4743 0,01/01/2011,31/12/2011,,../data/output/scraped_data_20180210-3.csv,2011-allyear
399553,Ministerio del Interior y Seguridad Pública,Gobernación de Los Andes,per_contrata,2011,allyear,4,profesional,lemus,guerra,ANA DEL CARMEN,,PERIODISTA,JEFE DE GABINETE,5,1,Pesos,1.275.035,No,4743 0,01/01/2011,31/12/2011,,../data/output/scraped_data_20180210-3.csv,2011-allyear
399554,Ministerio del Interior y Seguridad Pública,Gobernación de Los Andes,per_contrata,2011,allyear,5,profesional,araya,maldonado,DANILO RAMON,,INGENIERO DE EJECUCION EN MECANICA,ASESOR AUTORIDAD,5,2,Pesos,1.176.027,No,4743 1,01/01/2011,31/12/2011,,../data/output/scraped_data_20180210-3.csv,2011-allyear
399555,Ministerio del Interior y Seguridad Pública,Gobernación de Los Andes,per_contrata,2011,allyear,6,profesional,diaz,araya,RODRIGO MIGUEL,,LICENCIADO EN CIENCIAS DE LA INVESTIGACION CR...,ASISTENTE DE COORDINADOR EN DEPENDENCIAS DEL ...,5,1,Pesos,1.169.596,No,4743 0,01/01/2011,31/12/2011,,../data/output/scraped_data_20180210-3.csv,2011-allyear
399556,Ministerio del Interior y Seguridad Pública,Gobernación de Los Andes,per_contrata,2011,allyear,7,tecnico,aboal,madriaga,JOSE LIONEL,,LICENCIA DE ENSEÑANZA MEDIA,TECNICO,5,1,Pesos,531.461,No,4743 0,01/01/2011,31/12/2011,,../data/output/scraped_data_20180210-3.csv,2011-allyear
399557,Ministerio del Interior y Seguridad Pública,Gobernación de Los Andes,per_contrata,2011,allyear,8,tecnico,alamos,guerra,CARLOS FRANCISCO,,LICENCIA DE ENSEÑANZA MEDIA,ENCARGADO DE EMERGENCIAS,5,1,Pesos,476.993,No,4743 0,01/01/2011,31/12/2011,,../data/output/scraped_data_20180210-3.csv,2011-allyear
399558,Ministerio del Interior y Seguridad Pública,Gobernación de Los Andes,per_contrata,2011,allyear,9,administrativo,lopez,nuñez,ANA MARIA,,SECRETARIA ADMINISTRATIVA,ADMINISTRATIVO ADMINISTRACION,5,2,Pesos,496.686,No,4743 10,01/01/2011,31/12/2011,,../data/output/scraped_data_20180210-3.csv,2011-allyear
399559,Ministerio del Interior y Seguridad Pública,Gobernación de Los Andes,per_contrata,2011,allyear,10,administrativo,garay,riquelme,ALEJANDRO ALFONSO,,LICENCIA DE ENSEÑANZA MEDIA,ENCARGADO DE EXTRANJERIA,5,2,Pesos,440.370,No,4743 4,01/01/2011,31/12/2011,,../data/output/scraped_data_20180210-3.csv,2011-allyear


In [57]:
dfscraped['yearmonth'].value_counts()

2008-allyear                                                48627
2007-allyear                                                45263
2011-allyear                                                42654
2010-allyear                                                42307
2006-allyear                                                41761
2014-allyear                                                39372
2012-allyear                                                38411
2013-allyear                                                38275
2009-allyear                                                36501
2015-allyear                                                36045
2016-allyear                                                28962
2017-allyear                                                25570
2017-Octubre                                                20073
2017-Septiembre                                             19698
2016-Julio                                                  19615
2016-Agost

# Qualifications Analysis

## Cleaning and Standardizing Data 

In [77]:
dfscraped1['qualif1'] = dfscraped['qualif'].str.title()

In [78]:
dfscraped1['qualif1'].value_counts()

Abogado                                                                                                                             87296
Enseñanza Media                                                                                                                     82125
Contador Auditor                                                                                                                    57898
Ingeniero Comercial                                                                                                                 54878
Asistente Social                                                                                                                    53551
Licencia De Enseñanza Media                                                                                                         45465
Licencia Enseñanza Media                                                                                                            41448
Arquitecto                        

# Cleaning SENAME

In [35]:
dfscraped1.entity.value_counts(dropna=False)

Ministerio de Obras Públicas                       297422
Ministerio del Interior y Seguridad Pública        282969
Ministerio de Hacienda                             194651
Ministerio de Justicia y Derechos Humanos          181424
Ministerio de Educación                            179805
Ministerio de Vivienda y Urbanismo                 167945
Ministerio de Economía, Fomento y Turismo          132740
Ministerio del Trabajo y Previsión Social          125418
Ministerio de Desarrollo Social                    106861
Consejo de Defensa del Estado                       57967
Ministerio de Defensa Nacional                      47427
Ministerio de Minería                               45374
Ministerio de Agricultura                           44591
Ministerio de Relaciones Exteriores                 25464
Ministerio Secretaría General de la Presidencia     14657
Presidencia de la República                         12791
Ministerio del Medio Ambiente                        7224
Ministerio del

In [36]:
dfjusticia = dfscraped1.loc[dfscraped1.entity == 'Ministerio de Justicia y Derechos Humanos']

In [37]:
dfjusticia.department.value_counts()

Servicio Nacional de Menores (SENAME)                  132412
per_contrata                                            38030
Servicio de Registro Civil e Identificación (SRCEI)      6102
Defensoría Penal Pública                                 3316
per_planta                                               1564
Name: department, dtype: int64

In [39]:
dfsename = dfjusticia.loc[dfjusticia.department == 'Servicio Nacional de Menores (SENAME)']

In [40]:
dfsename.groupby(['year', 'month'])['entity'].count()

year        month  
ABRIL       allyear    8213
AGOSTO      allyear    8601
Año 2006    allyear    2806
Año 2007    allyear    3185
Año 2008    allyear    3387
Año 2009    allyear    3499
Año 2010    allyear    3985
Año 2011    allyear    4017
Año 2012    allyear    4062
Año 2013    allyear    4831
Año 2014    allyear    4379
Año 2015    allyear    6035
Año 2016    allyear      47
DICIEMBRE   allyear    8576
ENERO       allyear    7990
FEBRERO     allyear    8113
JULIO       allyear    8476
JUNIO       allyear    8367
MARZO       allyear    8178
MAYO        allyear    8282
NOVIEMBRE   allyear    4190
OCTUBRE     allyear    4534
SEPTIEMBRE  allyear    8659
Name: entity, dtype: int64

## Year label is a month + it is Known to be from a certain year 

In [ ]:
print(dfsename.shape)
# Monthly data, set define months
dfmonths = pd.read_csv('./data/months.csv', dtype={'month2' : str, 'month3' : int})
dfmonths.month3 = dfmonths.month3.astype(int)

months = list(dfmonths.month2.values)

# all the rows that are months we know they are 2016
# mark them as this transformation
dfsename.ix[:,'TmonthIsKnownYear'] = 0
dfsename.ix[dfsename.year.str.lower().isin(months), 'TmonthIsKnownYear'] = 1

# Write Months
dfsename.ix[dfsename.TmonthIsKnownYear == 1, 'month2'] = dfsename.year.str.lower()
dfsename.ix[dfsename.TmonthIsKnownYear == 0, 'month2'] = 'allyear'
dfsename = pd.merge(dfsename, dfmonths, how='left', on='month2')

# Write years
dfsename.ix[:, 'year2'] = dfsename.year.str.extract('( [0-9]*)$', expand=False)
dfsename.ix[dfsename.TmonthIsKnownYear == 1, 'year2'] = 2016
dfsename.year2 = pd.to_numeric(dfsename.year2)

# Include Rows Flag
dfsename['include'] = 1
dfsename.ix[(dfsename.year2 == 2016) & (dfsename.month2 == 'allyear'), 'include'] = 0 # year 2016
dfsename.ix[(dfsename.year == 0) | (dfsename.year == 1), 'include'] = 0 # zeroes and ones

print(dfsename.shape)

In [ ]:
print(dfsename.replace(np.nan, -1).groupby(['include' ,'TmonthIsKnownYear', 'year', 'year2', 'month2', 'month3'])['entity'].count())

## Expand yearly data to months. Allows filtering people from months by start_date and end_date.

In [ ]:
print(dfsename.shape)

# Define what df we want to expand to months
cols = [col for col in dfsename.columns if ((col !='month2') and (col != 'month3') )]
df_to_monthly = dfsename.ix[dfsename.month2 == 'allyear', cols]
print(df_to_monthly.shape)

# Keys to mege
df_to_monthly['key'] = 1
dfmonths['key'] = 1

# Generate cartesian product
dfcartesian = pd.merge(df_to_monthly, dfmonths, on='key')
cols2 = [col for col in dfcartesian.columns if col !='key']
dfcartesian = dfcartesian.ix[:,cols2]
print(dfcartesian.shape)

# Remove yearly rows
dfsename = dfsename.ix[dfsename.month2 != 'allyear', :]

# Add monthly rows
dfsename = pd.concat([dfsename, dfcartesian])

In [ ]:
print(dfsename.shape)
print(dfsename.replace(np.nan, -1).groupby(['include' ,'TmonthIsKnownYear', 'year2', 'month3'])['entity'].count().unstack('year2'))

In [ ]:
# Set the datetimeindex index
dfsename['curDate'] = dfsename.year2.map(int).map(str) + '-' + dfsename.month3.map(int).map(str)
dfsename['curDate2'] = pd.to_datetime(dfsename.curDate, format='%Y-%m')
dfsename = dfsename.set_index(pd.DatetimeIndex(dfsename['curDate2']))


In [ ]:
dfsename.ix[dfsename.include == 1].replace(np.nan, -1).groupby('contract').resample('M')['curDate2'].count().unstack('contract').plot()

Year 2016 looks weird because we have the actual monthly data. The rest of the years are just unparsed yearly data.

In [ ]:
dfsename['start_date'] = dfsename['start_date'].astype(str)
dfsename['end_date'] = dfsename['end_date'].astype(str)

# Fix some guys
dfsename.start_date = dfsename.start_date.str.replace('28/10/201$', '28/10/2010')

# Transform
dfsename['start_date2'] = pd.to_datetime(dfsename['start_date'], format='%d/%m/%Y', errors='coerce')
dfsename['end_date2'] = pd.to_datetime(dfsename['end_date'], format='%d/%m/%Y', errors='coerce')


In [ ]:
print('Start nulls\n', dfsename.ix[pd.isnull(dfsename.start_date2), 'start_date'].value_counts())
print('End nulls\n', dfsename.ix[pd.isnull(dfsename.end_date2), 'end_date'].value_counts())

In [ ]:

# Finished before the report date and not indefinite contract
dfsename.ix[(dfsename.end_date2 < dfsename.curDate2 + timedelta(days=14)) & (dfsename.end_date != 'Indefinido') , 'include'] = 0
# Started after the report date
dfsename.ix[dfsename.start_date2 > dfsename.curDate2 + timedelta(days=14), 'include'] = 0
# Null values
dfsename.ix[dfsename.start_date2 == 'nan', 'include'] = 0
dfsename.ix[dfsename.start_date2 == '', 'include'] = 0


In [ ]:
dfsename.ix[dfsename.include == 1].replace(np.nan, -1).groupby('contract').resample('M')['curDate2'].count().unstack('contract').plot()

In [ ]:
# Percentage of included
included_pct = pd.concat([dfsename.groupby(['include', 'curDate2'])['entity'].count().unstack('include'),
                         dfsename.groupby(['include', 'curDate2'])['entity'].count().unstack('include').sum(axis=1)], axis=1, join='inner')
included_pct.columns = ['no', 'yes', 'total']
included_pct['no_pct'] = included_pct.no / included_pct.total
list(included_pct.ix[included_pct.no_pct > 0.5].index)

In [ ]:
dfsename.ix[(dfsename.curDate2 == '2007-01-01') & (dfsename.include == 0), 'start_date'].value_counts().head()

In [ ]:
# ASSUMPTION: We will assume that the start_date 01/06/2007 is wrong and replace by 01/01/2007
dfsename['TcorrectedDate'] = 0
dfsename.ix[dfsename.start_date == '01/06/2007', 'TcorrectedDate'] = 1
dfsename.ix[dfsename.start_date == '01/06/2007', 'start_date2'] = datetime.strptime('01/01/2007', '%d/%m/%Y')
dfsename.ix[(dfsename.start_date2 <= dfsename.curDate2 + timedelta(days=14)) & 
            ((dfsename.end_date2 >= dfsename.curDate2 + timedelta(days=14)) |
            (dfsename.end_date == 'Indefinido')) , 'include'] = 1


In [ ]:
dfsename.ix[dfsename.include == 1].replace(np.nan, -1).groupby('contract').resample('M')['curDate2'].count().unstack('contract').plot()

Looking more reasonable, there are some strange spikes at the end of 2007, 2008 and 2009.

In [ ]:
dfsename.ix[dfsename.include == 1].replace(np.nan, -1).groupby('contract').resample('M')['curDate2'].count().unstack('contract')

## Can we translate the count into value?

In [ ]:
dfsename['salary2'] = dfsename['salary'].replace('\.', '', regex=True)
dfsename.salary2 = dfsename.salary2.astype(float)
dfsename.salary2.value_counts().head()

* There are too many zeroes, checking.
* There are some values that are far too popular, checking source too

In [ ]:
dfsename.ix[dfsename.salary2 == 0].groupby(['curDate2'])['entity'].count().sort_index()

In [ ]:
dfsename.ix[(dfsename.salary2 == 0) & (dfsename.curDate2 == datetime(year=2014, month=7, day=1)), 'url'].value_counts()

The zeroes in 2014 and over are because of people who resigned

In [ ]:
dfsename.columns

In [ ]:
dfsename.ix[dfsename.salary2 == 1580920.0].groupby(['curDate2', 'givenn', 'lastn', 'lastn2' ])['entity'].count()

This popular salary is spread across several people

## There seems to be duplicated names. Checking

In [ ]:
dfsename.ix[(dfsename.givenn == 'miguel angel') & (dfsename.include == 1) & (dfsename.curDate2 == datetime(year=2016, month=6, day=1))].to_csv('./output/doublename.csv')#

In [ ]:
dfsename.ix[dfsename.include == 1].groupby(['curDate2', 'givenn', 'lastn', 'lastn2'])['entity'].count().sort_values(ascending=False)

In [ ]:
dfsename.ix[(dfsename.givenn == 'miguel angel') & (dfsename.lastn2 == 'canales') & (dfsename.include == 1) & (dfsename.curDate2 == datetime(year=2016, month=6, day=1))].to_csv('./output/doublename.csv')#

In [ ]:
dfscraped.ix[(dfscraped.givenn.str.lower() == 'miguel angel') & (dfscraped.lastn2.str.lower() == 'canales') & (dfscraped.end_date == 'Indefinido')].to_csv('./output/doublename1.csv')#

In [ ]:
dfscraped.ix[[714977, 1705528]]